## 1. Settings of Notebook

In [ ]:
URL = "http://localhost:3002"


## 2. Loading of materials

In [ ]:
material_json = {
    "name": "New Material",
    "basis": {
        "elements": [
            {"id": 0, "value": "O"},
            {"id": 1, "value": "C"}
        ],
        "coordinates": [
            {"id": 0, "value": [0, 0, 0]},
            {"id": 1, "value": [0.25, 0.25, 0.5]}
        ],
        "units": "crystal",
        "cell": [
            [3.34892, 0, 1.9335],
            [1.116307, 3.157392, 1.9335],
            [0, 0, 3.867]
        ],
        "constraints": []
    },
    "lattice": {
        "a": 3.867, "b": 3.867, "c": 3.867,
        "alpha": 60, "beta": 90, "gamma": 90,
        "units": {"length": "angstrom", "angle": "degree"},
        "type": "FCC",
        "vectors": {
            "a": [3.34892, 0, 1.9335],
            "b": [1.116307, 3.157392, 1.9335],
            "c": [0, 0, 3.867],
            "alat": 1,
            "units": "angstrom"
        }
    },
    "isNonPeriodic": False
}

materials_settings = [{
    "material_json": material_json,
    "name": "New Material.gif"
}]

## 3. Definitions

In [ ]:
from IPython.display import display, IFrame, Javascript
import json

from utils.visualize import visualize_materials


def send_message_to_iframe(message):
    """
    Creates and displays an iframe pointing to `url`,
    then sends `message` to the iframe via postMessage.
    """
    # Create and display iframe


    # Prepare JavaScript code to send message to iframe
    js_code = f"""
    (function() {{
        // Wait for the iframe to be available
        const iframe = document.querySelector('iframe');
        if (iframe && iframe.contentWindow) {{
            setTimeout(() => {{
                try {{
                    iframe.contentWindow.postMessage({json.dumps(message)}, '*');
                    console.log('Message sent to iframe');
                }} catch (error) {{
                    console.error('Error sending message:', error);
                }}
            }}, 1000);  // slight delay to ensure iframe is ready
        }} else {{
            console.error('Iframe not found or not ready');
        }}
    }})();
    """
    display(Javascript(js_code))


def set_material_in_iframe(material_json):
    """
    Uses send_message_to_iframe to send a material configuration
    under the "material" key to the iframe at `url`.
    """
    send_message_to_iframe({"material": material_json})


def record_gif(filename, rotation_speed=60, frame_duration=0.05):
    """
    Uses send_message_to_iframe to send a message to the iframe at `url`
    to start recording a GIF of the visualization with the specified parameters.
    """
    message = {
        "action": "handleStartGifRecording",
        "parameters": [filename, rotation_speed, frame_duration]
    }
    send_message_to_iframe(message)

def set_camera( pos=(0, 5, 0), target=(0, 0, 0)):
    func_str = f"(wave) => {{ wave.camera.position.set({pos[0]}, {pos[1]}, {pos[2]}); wave.camera.lookAt({target[0]}, {target[1]}, {target[2]}); wave.render(); }}"
    message = {
        "action": "doFunc",
        "parameters": [func_str]
    }
    send_message_to_iframe(message)

def toggle_bonds():
    message = {
        "action": "handleToggleBonds",
        "parameters": [None]
    }
    send_message_to_iframe(message)



## 4. Gif Generation

In [ ]:
# Example usage with a material configuration:
for material_settings in materials_settings:
    GIF_NAME =  material_settings["name"] or material_settings["material_json"]["name"]
    iframe = IFrame(src=URL, width="800", height="600")
    display(iframe)
    # Send the material configuration to the iframe at the specified URL
    set_material_in_iframe(material_json)
    toggle_bonds()
    set_camera()
    record_gif( GIF_NAME, 60, 0.05)
